Para el ajax en enlima.pe https://www.enlima.pe/calendario-cultural/dia/2020-01-14?page=1

In [56]:
import requests 
from bs4 import BeautifulSoup

import datetime
from datetime import timedelta

import re

In [2]:
year = 2018
month = 1
day = 1
time_obj = datetime.datetime(year, month, day, 0, 0)
i=1
date_dict={}
while time_obj.year < 2019:
    date_dict[i]={}
    date_dict[i] = {}
    date_dict[i]['date'] = i
    date_dict[i]["year:"] = time_obj.strftime("%Y")
    date_dict[i]["month:"] = time_obj.strftime("%m")
    date_dict[i]["day:"] = time_obj.strftime("%d")
    date_dict[i]["Start time:"] = time_obj.strftime("%H:%M:%S")
    time_obj += timedelta(minutes=59)
    time = time_obj.strftime("%H:%M:%S")
    time_obj += timedelta(minutes=1)
    date_dict[i]["End time:"] = time
    i+=1
print(date_dict)

{1: {'date': 1, 'year:': '2018', 'month:': '01', 'day:': '01', 'Start time:': '00:00:00', 'End time:': '00:59:00'}, 2: {'date': 2, 'year:': '2018', 'month:': '01', 'day:': '01', 'Start time:': '01:00:00', 'End time:': '01:59:00'}, 3: {'date': 3, 'year:': '2018', 'month:': '01', 'day:': '01', 'Start time:': '02:00:00', 'End time:': '02:59:00'}, 4: {'date': 4, 'year:': '2018', 'month:': '01', 'day:': '01', 'Start time:': '03:00:00', 'End time:': '03:59:00'}, 5: {'date': 5, 'year:': '2018', 'month:': '01', 'day:': '01', 'Start time:': '04:00:00', 'End time:': '04:59:00'}, 6: {'date': 6, 'year:': '2018', 'month:': '01', 'day:': '01', 'Start time:': '05:00:00', 'End time:': '05:59:00'}, 7: {'date': 7, 'year:': '2018', 'month:': '01', 'day:': '01', 'Start time:': '06:00:00', 'End time:': '06:59:00'}, 8: {'date': 8, 'year:': '2018', 'month:': '01', 'day:': '01', 'Start time:': '07:00:00', 'End time:': '07:59:00'}, 9: {'date': 9, 'year:': '2018', 'month:': '01', 'day:': '01', 'Start time:': '0

#### To do: 

- Una función a la que le dé dos fechas y me bote una lista con todos los días dentro de esa fecha. 

- Una función a la que le pase la lista de fechas y me scrapee todos los links que hay dentro de esas fechas. 

- Una función última de scrapper que pueda jalarme toda la información de las webs. El formato será tabular y en csv. (avanzar hasta tener un mvp e identificar los campos que tendré. También scrapear links)  **Avanzado a un 80%**

In [3]:
# Falta hacer los 
def making_links(fecha):
    fecha = '2020-01-14'
    link = 'https://www.enlima.pe/calendario-cultural/dia/'+fecha+'?page=1'

In [4]:

link = 'https://www.enlima.pe/calendario-cultural/dia/'+'2020-01-14'+'?page=1' 
response = requests.get(link)
soup = BeautifulSoup(response.text,"html.parser")

In [5]:
lista_links_internos = []
for i in soup.find('div',{'class':'innera'}).find_all('td',{'class':'views-field views-field-title views-align-left'}):
    link_interno = [a['href'] for a in i.find_all('a',href=True)]
    lista_links_internos.append(link_interno)

In [6]:
dict_info = {}
for i in lista_links_internos:
    link_entero = 'https://www.enlima.pe' + str(i[0])
    

In [72]:
dict_scrap_final = {}
i=1
def scrap_me_all(link_interno):
    global dict_scrap_final
    dict_scrap_interno = {}
    response = requests.get(link_interno)
    soup = BeautifulSoup(response.text,"html.parser")
    titulo = soup.find('h1',{'class':'page__title title'}).text
    basic_info = [div.text for div in soup.find_all('div',{'class':'field-item even'})[2:7]]
    tipo, lugar, intervalo_fecha, precio, descripcion_basica = map(str,basic_info)
    to_parse = [elem.text for elem in soup.find('div',{'class':
                                                       'ds-1col node node-lugar view-mode-reference clearfix'}).find_all('div',
                                                                                                                         {'class':'field-item even'})]
    pattern = re.compile(r'-\d*.\d*,-\d*.\d*',re.MULTILINE|re.DOTALL) #Tengo el patron de las coordenadas
    script = soup.find_all('script',text=pattern) #saco el script que contiene el patron
    coordenadas_list = re.findall(pattern,str(script))[0].split(',') #Lo obtengo y lo pongo dentro de una lista. 
    dict_scrap_final['evento'+str(i)] = {'nombre_evento':titulo,'tipo_evento':tipo,'lugar_evento':lugar,
                                        'intervalo_fecha':intervalo_fecha,'precio':precio,
                                         'descripcion_basica':descripcion_basica,'data_extra':to_parse,
                                         'coordenadas':coordenadas_list}

In [73]:
scrap_me_all('https://www.enlima.pe/agenda-cultural/exposicion/calendarios-japoneses')

In [74]:
dict_scrap_final

{'evento1': {'nombre_evento': 'Calendarios japoneses',
  'tipo_evento': 'Centro Cultural Peruano Japonés',
  'lugar_evento': '9:00 am',
  'intervalo_fecha': 'Del 06.01.2020 al 31.01.2020',
  'precio': 'GRATIS',
  'descripcion_basica': 'La Embajada del Japón en el Perú y la Asociación Peruano Japonesa presentan esta tradicional muestra de almanaques japoneses. La muestra reúne ejemplares con bellos diseños basados en las tradiciones japonesas, que los apartan de su función utilitaria para convertirlos en objetos artísticos de gran atractivo.\nEsta exposición se puede visitar del 6 al 31 de enero 2020, todos los días de 9 am a 9 pm.\nINGRESO LIBRE\n',
  'data_extra': ['Centro Cultural Peruano Japonés',
   'Calle Gregorio Escobedo 803, Residencial San Felipe',
   'Jesús María',
   'lun-dom 9am-9pm\n',
   '518-7450',
   '+ info',
   ''],
  'coordenadas': ['-77.055456899999995812322595156729221343994140625',
   '-12.08798110000000036734491004608571529388427734375']}}

In [44]:
response = requests.get('https://www.enlima.pe/agenda-cultural/exposicion/calendarios-japoneses')
soup = BeautifulSoup(response.text,"html.parser")

In [45]:
#Titulo
soup.find('h1',{'class':'page__title title'}).text

'Calendarios japoneses'

In [46]:
# tipo, lugar, intervalo_fecha, precio, descripcion_basica
basic_info = [div.text for div in soup.find_all('div',{'class':'field-item even'})[2:7]]
tipo, lugar, intervalo_fecha, precio, descripcion_basica = map(str,basic_info)

In [49]:
print(tipo,'--', lugar,'--', intervalo_fecha,'--', precio,'--', descripcion_basica)

Centro Cultural Peruano Japonés -- 9:00 am -- Del 06.01.2020 al 31.01.2020 -- GRATIS -- La Embajada del Japón en el Perú y la Asociación Peruano Japonesa presentan esta tradicional muestra de almanaques japoneses. La muestra reúne ejemplares con bellos diseños basados en las tradiciones japonesas, que los apartan de su función utilitaria para convertirlos en objetos artísticos de gran atractivo.
Esta exposición se puede visitar del 6 al 31 de enero 2020, todos los días de 9 am a 9 pm.
INGRESO LIBRE



In [50]:
# Direccion, Distrito, horarios y dias, numero
#OJO, no todos tienen estos campos. Por eso lo pondré para parsear luego. Que me encuentre todo lo que pueda. 
to_parse = [elem.text for elem in soup.find('div',{'class':'ds-1col node node-lugar view-mode-reference clearfix'}).find_all('div',{'class':'field-item even'})]
to_parse

['Centro Cultural Peruano Japonés',
 'Calle Gregorio Escobedo 803, Residencial San Felipe',
 'Jesús María',
 'lun-dom 9am-9pm\n',
 '518-7450',
 '+ info',
 '']

In [69]:
#Longitud y latitud
# Algunos tienen otros no. Entonces se recomienda hacer un try except... probar con los que no tienen y sí. 
import re

pattern = re.compile(r'-\d*.\d*,-\d*.\d*',re.MULTILINE|re.DOTALL) #Tengo el patron de las coordenadas

script = soup.find_all('script',text=pattern) #saco el script que contiene el patron

coordenadas_list = re.findall(pattern,str(script))[0].split(',') #Lo obtengo y lo pongo dentro de una lista. 

In [70]:
coordenadas_list

['-77.055456899999995812322595156729221343994140625',
 '-12.08798110000000036734491004608571529388427734375']